In [0]:
import torch
from torchtext import data
from torchtext import datasets
import torch.optim as optim
import time
import spacy
import torch.nn as nn


torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [8]:
train, test = datasets.IMDB.splits(TEXT, LABEL)


downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:09<00:00, 9.13MB/s]


In [0]:
train, valid = train.split()


In [11]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train)

.vector_cache/glove.6B.zip: 862MB [06:31, 2.20MB/s]                          
 99%|█████████▉| 397762/400000 [00:15<00:00, 24482.21it/s]

In [0]:
class RNN(nn.Module):
    def __init__(self, vocab_size, emb_dim, h_dim, out_dim, layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(emb_dim, 
                           h_dim, 
                           num_layers=layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(h_dim * 2, out_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, length):
        
        
        embed = self.dropout(self.embedding(text))
        
        p_embed = nn.utils.rnn.pack_padded_sequence(embed, length)
        
        p_output, (hidden, cell) = self.rnn(p_embed)
        
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(p_output)

        # apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
            
        return self.fc(hidden)

In [0]:
in_dim = len(TEXT.vocab)
emb_dim = 100
h_dim = 256
out_dim = 1
layers = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(in_dim, emb_dim, h_dim, out_dim, layers, BIDIRECTIONAL, DROPOUT, PAD_IDX)

In [18]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print('The model has {} trainable parameters'.format(count_parameters(model)))

The model has 4810857 trainable parameters


In [19]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [20]:
model.embedding.weight.data.copy_(pretrained_embeddings)


tensor([[-1.3183,  0.5577,  0.2282,  ..., -2.1676, -0.7688, -0.8385],
        [ 1.1523, -0.9973, -0.4263,  ...,  0.2774, -0.5144, -0.0841],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-1.4093, -1.8312,  0.0044,  ...,  2.1618, -0.5821,  0.5371],
        [ 0.4222, -1.0156,  0.7841,  ..., -0.2915,  0.8334, -0.4019],
        [ 0.7090,  1.5195,  0.3851,  ...,  0.1215,  0.4930, -0.5784]])

In [21]:

model.embedding.weight.data[TEXT.vocab.stoi[TEXT.unk_token]] = torch.zeros(emb_dim)
model.embedding.weight.data[PAD_IDX] = torch.zeros(emb_dim)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-1.4093, -1.8312,  0.0044,  ...,  2.1618, -0.5821,  0.5371],
        [ 0.4222, -1.0156,  0.7841,  ..., -0.2915,  0.8334, -0.4019],
        [ 0.7090,  1.5195,  0.3851,  ...,  0.1215,  0.4930, -0.5784]])


In [0]:
optimizer = optim.Adam(model.parameters())

In [36]:
criterion = nn.BCEWithLogitsLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size = 64,
    sort_within_batch = True,
    device = device)
model = model.to(device)
criterion = criterion.to(device)
print(device)

cuda


In [0]:
def binary_accuracy(predictions, y):
    rounded = torch.round(torch.sigmoid(predictions))
    correct = (rounded == y).float() 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def Train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [41]:
current_loss = float('inf')

for epoch in range(5):

    start_t = time.time()
    
    train_loss, train_acc = Train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_t = time.time()

    epoch_mins, epoch_secs = epoch_time(start_t, end_t)
    
    if valid_loss < current_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'SecondModel.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 40s
	Train Loss: 0.673 | Train Acc: 58.38%
	 Val. Loss: 0.666 |  Val. Acc: 60.75%
Epoch: 02 | Epoch Time: 0m 40s
	Train Loss: 0.686 | Train Acc: 54.48%
	 Val. Loss: 0.680 |  Val. Acc: 58.96%
Epoch: 03 | Epoch Time: 0m 40s
	Train Loss: 0.649 | Train Acc: 61.60%
	 Val. Loss: 0.620 |  Val. Acc: 66.79%
Epoch: 04 | Epoch Time: 0m 40s
	Train Loss: 0.532 | Train Acc: 73.58%
	 Val. Loss: 0.437 |  Val. Acc: 80.25%
Epoch: 05 | Epoch Time: 0m 40s
	Train Loss: 0.478 | Train Acc: 77.43%
	 Val. Loss: 0.468 |  Val. Acc: 78.90%


In [42]:
model.load_state_dict(torch.load('SecondModel.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.470 | Test Acc: 78.79%
